<a href="https://colab.research.google.com/github/AWH-GlobalPotential-X/AWH-Geo/blob/master/notebooks/Figure_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to AWH-Geo Threshold Processor

This tool requires a Google Drive and Earth Engine Account

Click "Connect" at the top right of this notebook.

Then run each of the code blocks below, following instructions.

In [ ]:
!pip install sciplotlib SciencePlots
!apt install texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra cm-super dvipng

In [ ]:
import matplotlib.pyplot as plt
import mizani
import numpy as np
import pandas as pd
import plotnine as gg
from matplotlib import colors
from sciplotlib import style as spstyle
from sciplotlib import polish as sppolish
from scipy import interpolate

from IPython.display import display

# Operating Hours per GHI

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
# Load from Drive
df = pd.read_csv('gdrive/My Drive/results_fc_cumul.csv')
df.head()

In [ ]:
df = df.rename(columns={
    'ghi_threshold': 'GHI',
    'opHours': 'opHr',
    'rh_threshold': 'relHumidity',
    'mktSize': 'mktSize',
})
df.head()

In [ ]:
dfp = df.copy()
dfp['index'] = 'GHI:' + df['GHI'].map(str) + '_opHr:' + df['opHr'].map(str)

# Limit the dataset to specific opHr and GHI
dfp = dfp.query('opHr >= 1 and opHr <= 8 and (GHI == 400 or GHI == 600)')

# Retrieve the ribbon min value
dfp = dfp.sort_values('opHr')
dfp['mktSize_min'] = dfp.groupby(['GHI', 'relHumidity']).mktSize.shift(-1)
dfp['mktSize_min'] = dfp.mktSize_min.fillna(0)
dfp.head()

In [ ]:
# Change this to False|True to reverse the use of colors
opHr_ascending_order = False

# Fix the rendering order of opHr curves with overlaps
df_ghi = dfp.copy()
df_ghi['opHr'] = pd.Categorical(df_ghi.opHr, categories=df_ghi.opHr.sort_values(ascending=opHr_ascending_order).unique().tolist())

# Change market size unit to million
df_ghi['mktSize'] = df_ghi.mktSize.values / 1e6
df_ghi['mktSize_min'] = df_ghi.mktSize_min.values / 1e6

In [ ]:
p = (gg.ggplot(df_ghi,
               gg.aes(x='relHumidity',
                      y='mktSize',
                      group='index'))
    + gg.geom_ribbon(gg.aes(ymin='mktSize_min', ymax='mktSize', fill='factor(opHr)'), alpha=0.8)
    + gg.geom_line(gg.aes(color='factor(opHr)'), size=2, alpha=1.0, show_legend=False)
    + gg.facet_wrap('GHI', nrow=1, labeller=lambda x: f'GHI Threshold [W/m²]: {x}')
    + gg.scale_color_cmap_d()
    + gg.scale_fill_cmap_d()
    + gg.scale_x_continuous(breaks=range(10, 90, 10))
    + gg.scale_y_continuous(labels=mizani.formatters.comma_format())
    + gg.theme_minimal()
    + gg.theme(figure_size=(12, 6),
               text=gg.element_text(size=18),
               title=gg.element_text(size=24, weight='bold', margin={'b': 30.0}),
               axis_title=gg.element_text(size=18, weight='normal'),
               axis_title_x=gg.element_text(margin={'t': 15.0}),
               axis_title_y=gg.element_text(margin={'r': 15.0}),
               legend_position=(.80, -0.10),
               legend_direction='horizontal',
               legend_title=gg.element_text(size=18),
               legend_text=gg.element_text(size=18),
               legend_background=gg.element_rect(fill='#f8f8f8', color='#cccccc'))
    + gg.labs(title='Users reached by climate thresholds',
              fill='Operational Hours per Day',
              color=gg.element_blank(),
              x=r'rH Threshold [\%]',
              y='People [millions]'))

fig = p.draw()
fig.patch.set_facecolor('white')
for ax in fig.get_axes():
    ax.xaxis.set_tick_params(which='major', size=0, width=0.0, direction='in', top='on')
    ax.xaxis.set_tick_params(which='minor', size=0, width=0.0, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=0, width=0.0, direction='in', top='on')
    ax.yaxis.set_tick_params(which='minor', size=0, width=0.0, direction='in', top='on')

## Nature figure style

In [ ]:
%matplotlib inline

from collections import defaultdict

import matplotlib as mpl
from matplotlib import cm
from matplotlib import patheffects

plt.style.reload_library()
plt.style.use('science')


def make_plot_ghi(axes):
  # Define how many panels do we need
  facet_cols = df_ghi['GHI'].value_counts().shape[0]

  # Define numbers of colors from scale
  viridis = cm.get_cmap('viridis', 8)

  # Plot the data
  lines_handles = defaultdict(list)
  lines_labels = defaultdict(list)
  areas_handles = defaultdict(list)
  areas_labels = defaultdict(list)
  for ax_idx, (ghi_label, ghi_data) in enumerate(df_ghi.groupby('GHI')):
    ax = axes[ax_idx]
    #ax.set_title(f'GHI Threshold [W/m²]: {int(ghi_label)}')
    ax.set_title(f'GHI [W/m²]: {int(ghi_label)}', fontsize=8)
    for opHr_idx, (opHr_label, opHr_data) in enumerate(ghi_data.groupby('opHr')):
      opHr_data = opHr_data.sort_values('relHumidity')

      # Draw the area under the curve (ribbon style)
      line, = ax.plot(
          opHr_data['relHumidity'], opHr_data['mktSize'],
          linestyle='solid', linewidth=1.5, color=viridis(opHr_idx),
          label=opHr_label)
      lines_handles[ax_idx].append(line)
      lines_labels[ax_idx].append(opHr_label)

      # Draw the line
      area = ax.fill_between(
          opHr_data['relHumidity'], opHr_data['mktSize_min'], opHr_data['mktSize'],
          color=viridis(opHr_idx), alpha=0.8,
          label=opHr_label)
      areas_handles[ax_idx].append(area)
      areas_labels[ax_idx].append(opHr_label)

  for ax in axes:
    # Set the scale limits
    ax.set_xlim([10, 100])
    ax.set_ylim([0, 2500])

    # Hide the top and right spines of the axis
    ax.spines['right'].set_visible(True)
    ax.spines['top'].set_visible(True)
    for spine in ax.spines.values():
      spine.set_linewidth(1.0)

    # Edit the major and minor ticks of the x and y axes
    ax.xaxis.set_tick_params(which='major', size=5, width=1.0, direction='in', top='on')
    ax.xaxis.set_tick_params(which='minor', size=2, width=0.75, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=5, width=1.0, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=2, width=0.75, direction='in', right='on')

    # Format x-axis ticks
    ax.xaxis.set_major_locator(mpl.ticker.FixedLocator(np.arange(20, 120, 20)))
    ax.set_xticklabels(np.arange(20, 120, 20))

    # Format y-axis ticks
    ax.yaxis.set_major_formatter(
        mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

    # Add the x-axis abd y-axis labels
    ax_x_label = ax.set_xlabel(r'Relative Humidity Threshold [\%]', labelpad=10)
    ax.xaxis.set_label_coords(1.10, -0.20)
    ax.set_ylabel(r'People without SMDW [millions]', labelpad=10)

  # Hide y-axis title and labels on second plot
  axes[1].set_ylabel(None)
  axes[1].yaxis.set_ticklabels([])

  # Hide x-axis title on the second plot
  axes[1].set_xlabel(None)

  # Add the legend for linear lines
  #leg1 = axes[0].legend(
  #    lines_handles[0],
  #    lines_labels[0],
  #    bbox_to_anchor=(0.5, -0.30), loc='upper center', ncol=4,
  #    fontsize=6,
  #    handlelength=2,
  #    title=r'\ul{Operational Hours per Day}')

  # Add the legend for areas
  leg2 = axes[0].legend(
      areas_handles[1],
      areas_labels[1],
      bbox_to_anchor=(0.0, -0.40), loc='upper left', ncol=4,
      #fontsize=6,
      #handlelength=2,
      title=r'\ul{Operational Hours per Day}')
  
  # Manually add the first legend back
  #axes[0].add_artist(leg1)
  #axes[1].add_artist(leg2)


plt.rc('text.latex', preamble=r'\usepackage{soul}')
plt.rc('font', **{'family':'sans-serif', 'sans-serif': ['Arial']})
plt.rcParams["axes.axisbelow"] = False

with plt.style.context(['science', 'ieee', 'retro']):  # 'no-latex'
  fig, axes = plt.subplots(1, 2, figsize=(8, 4))
  make_plot_ghi(axes)
  fig.set_size_inches(6, 3)

fig.patch.set_facecolor('white')
fig.suptitle('User Reach by Climate Thresholds')

# Specific yields by curves

In [ ]:
logistic_params = {
    '2.0 bil. (logistic)': {
      'L': 4.8,  # L.kWh^-1
      'x0': 60,  # midpoint in percentage
      'k': 0.1,  # growth rate
    },
    # '1.5 bil. (logistic)': {
    #   'L': 3.3,  # L.kWh^-1
    #   'x0': 60,  # midpoint in percentage
    #   'k': 0.1,  # growth rate
    # },
    '1.0 bil. (logistic)': {
      'L': 2.4,  # L.kWh^-1
      'x0': 60,  # midpoint in percentage
      'k': 0.1,  # growth rate
    },
    '0.5 bil. (logistic)': {
      'L': 1.8,  # L.kWh^-1
      'x0': 60,  # midpoint in percentage
      'k': 0.1,  # growth rate
    }
}

linear_params = {
    '2.0 bil. (linear)': {
      'Max': 2.6,  # L.kWh^-1
      'a': 0.026,
      'b': 0.0,
    },
    # '1.5 bil. (linear)': {
    #   'Max': 2.2,  # L.kWh^-1
    #   'a': 0.022,
    #   'b': 0.0,
    # },
    '1.0 bil. (linear)': {
      'Max': 1.9,  # L.kWh^-1
      'a': 0.019,
      'b': 0.0,
    },
    '0.5 bil. (linear)': {
      'Max': 1.6,  # L.kWh^-1
      'a': 0.016,
      'b': 0.0,
    }
}

df_yields = pd.DataFrame({
    'rH': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'SMAG Zhao': [None, None, 0.21, None, None, 3.71, None, None, 9.28, None],
    'ZMW': [0.19, 0.24, 0.28, 0.33, 0.42, 0.44, 0.47, 0.5, 0.53, None],
    'Bagheri': [None, None, 0.27, None, None, 0.67, None, None, None, None],
    'MOFs': [None, 0.19, None, None, None, None, 0.84, None, None, None],
})

In [ ]:
df_tidy = pd.melt(df_yields,
        id_vars=['rH'],
        value_vars=['SMAG Zhao', 'ZMW', 'Bagheri', 'MOFs'],
        var_name='device',
        value_name='yield')

# Flag some yield to be shown on the graph with numbers
df_tidy['show_value'] = False
df_tidy.loc[df_tidy.device == 'MOFs', 'show_value'] = True
df_tidy.loc[df_tidy.device == 'SMAG Zhao', 'show_value'] = True

df_tidy = df_tidy.dropna()
df_tidy.head()

In [ ]:
df_fits_tidy = pd.DataFrame()
x = np.linspace(0, 100, num=200)

for name, params in logistic_params.items():
  df_fits = pd.DataFrame({'rH': x})
  df_fits['fit'] = name
  df_fits['yield'] = params['L']/(1 + np.exp(-params['k']*(x- params['x0'])))
  df_fits['fit_type'] = 'logistic'
  df_fits_tidy = pd.concat([df_fits_tidy, df_fits])

for name, params in linear_params.items():
  df_fits = pd.DataFrame({'rH': x})
  df_fits['fit'] = name
  df_fits['yield'] = params['a']*x + params['b']
  df_fits['fit_type'] = 'linear'
  df_fits_tidy = pd.concat([df_fits_tidy, df_fits])

df_fits_tidy.head()

In [ ]:
from itertools import cycle

p = (gg.ggplot(gg.aes(x='rH'))
    + gg.geom_point(df_tidy, gg.aes(y='yield', shape='device'),
                    color='black', size=4, alpha=0.6, show_legend=True)
    + gg.geom_line(df_fits_tidy, gg.aes(y='yield', color='fit', linetype='fit_type'),
                   size=1, show_legend=True)
    + gg.scale_y_log10(limits=(0.1, 10))
    + gg.scale_x_continuous(breaks=np.linspace(0, 100, num=11),
                            limits=(10, 100))
    + gg.scale_color_manual(['#4a86e8', '#4a86e8', '#ff0101', '#ff0101', '#35a754', '#35a754'])
    + gg.theme_minimal()
    + gg.theme(figure_size=(8, 8))
    + gg.labs(title='Specific Yield by curve [L/kWh]',
              x='relative humidity',
              y='Specific Yield [L/kWh]')
)

print(p)

## Nature figure style

In [ ]:
%matplotlib inline

from collections import defaultdict

import matplotlib as mpl
from matplotlib import cm
from matplotlib import patheffects

plt.style.reload_library()
plt.style.use('science')


def make_plot_specific_yields(ax):
  linetypes = ['dashed', 'solid']
  
  # Define colors
  viridis = cm.get_cmap('viridis', 4)

  # Draw lines for fitted curves
  lines_handles = defaultdict(list)
  lines_labels = defaultdict(list)
  for linetype_idx, fit_type in enumerate(['linear', 'logistic']):
    linetype = linetypes[linetype_idx]
    for fit_idx, (fit_label, group) in enumerate(df_fits_tidy.query('fit_type == @fit_type').groupby('fit')):
      color = viridis(1.1*fit_idx/3)
      line, = ax.plot(group['rH'], group['yield'], linestyle=linetype, color=color,
                      label=fit_label)
      lines_handles[fit_type].append(line)
      lines_labels[fit_type].append(fit_label)
  
  # Draw scatter plot of devices
  device_handles = []
  device_labels = []
  markers = cycle(['o', '*', '^', 'p'])
  for marker_idx, (device, group) in enumerate(df_tidy.groupby('device')):
    marker = next(markers)
    handle = ax.scatter(group['rH'], group['yield'], marker=marker, color='black', linewidth=0, alpha=0.75,
                         label=device)
    device_handles.append(handle)
    device_labels.append(device)

  # Add yield value as annotations
  for idx, row in df_tidy.query('show_value').iterrows():
    xy = (row['rH'], row['yield'])
    xytext = (0.0, -12.0)  # Defines an offset in points
    arrowprops = dict(arrowstyle='->', color='grey', linewidth=1)
    # NB: Add a stroke around the text to detach it from the curves
    text = ax.annotate(
        row['yield'], xy, xytext, textcoords='offset points', ha='center',
        path_effects=[patheffects.withStroke(linewidth=2, foreground="w")])

  # Set the log scale on the y-axis with limits
  ax.set_yscale('log')
  ax.set_xlim([10, 100])
  ax.set_ylim([1e-1, 10])

  # Hide the top and right spines of the axis
  ax.spines['right'].set_visible(True)
  ax.spines['top'].set_visible(True)
  for spine in ax.spines.values():
    spine.set_linewidth(1.0)

  # Edit the major and minor ticks of the x and y axes
  ax.xaxis.set_tick_params(which='major', size=5, width=1.0, direction='in', top='on')
  ax.xaxis.set_tick_params(which='minor', size=2, width=0.75, direction='in', top='on')
  ax.yaxis.set_tick_params(which='major', size=5, width=1.0, direction='in', right='on')
  ax.yaxis.set_tick_params(which='minor', size=2, width=0.75, direction='in', right='on')

  ax.yaxis.set_major_locator(mpl.ticker.FixedLocator([0.1, 0.5, 1, 5, 10]))
  ax.set_yticklabels([0.1, 0.5, 1, 5, 10])

  ax.xaxis.set_major_locator(mpl.ticker.FixedLocator(np.arange(20, 120, 20)))
  ax.set_xticklabels(np.arange(20, 120, 20))

  # Add the x-axis abd y-axis labels
  ax.set_xlabel(r'Relative Humidity [\%]', labelpad=10)
  ax.set_ylabel(r'Specific Yield [$L/kWh$] (log scale)', labelpad=10)

  # Add the legend for linear lines
  leg1 = ax.legend(lines_handles['linear'] + lines_handles['logistic'],
                   lines_labels['linear'] + lines_labels['logistic'],
                   bbox_to_anchor=(0.5 - 0.25, -0.24), loc='upper center', ncol=2,
                   prop={'size': 8},
                   title=r'\ul{Target (ppl without SMDW)}')
  #leg1 = ax.legend(lines_handles['linear'], lines_labels['linear'], bbox_to_anchor=(-0.02, -0.20), loc=2,
  #                 title=r'\ul{Linear Curve}')

  # Add the legend for logistic lines
  #leg2 = ax.legend(lines_handles['logistic'], lines_labels['logistic'], bbox_to_anchor=(0.30, -0.20), loc=2,
  #                 title=r'\ul{Logistic Curve}')
  
  # Add the legend for devices
  leg3 = ax.legend(device_handles, device_labels,
                   bbox_to_anchor=(0.5 + 0.35, -0.24), loc='upper center',
                   prop={'size': 8},
                   title=r'\ul{Experimental Results}')
  
  # Manually add the first legend back
  ax.add_artist(leg1)
  #ax.add_artist(leg2)
  ax.add_artist(leg3)


plt.rc('text.latex', preamble=r'\usepackage{soul}')
plt.rc('font', **{'family':'sans-serif', 'sans-serif': ['Arial']})
plt.rcParams["axes.axisbelow"] = False

with plt.style.context(['science', 'ieee', 'retro']):  # 'no-latex'
  fig, ax = plt.subplots()
  make_plot_specific_yields(ax)
  fig.set_size_inches(4, 3)

fig.patch.set_facecolor('white')
fig.suptitle('User Reach by Specific Yield')

# Output Rate by AWH

In [ ]:
# Load from Drive
df_output_rate = pd.read_csv('gdrive/My Drive/Pop_noSMDW_byOutput_fig4c.csv')
df_output_rate = df_output_rate.drop(columns=['system:index', '.geo']).set_index('0')
df_output_rate = df_output_rate.transpose().reset_index()
df_output_rate = df_output_rate.rename(columns={'index': 'yield'})
df_output_rate['yield'] = df_output_rate['yield'].astype(np.int64) / 100
df_output_rate.head()

## Nature figure style

In [ ]:
lines_labels = df_output_rate.columns.tolist()
if 'yield' in lines_labels:
  lines_labels.remove('yield')
lines_labels

In [ ]:
%matplotlib inline

from collections import defaultdict

import matplotlib as mpl
from matplotlib import cm
from matplotlib import patheffects

plt.style.reload_library()
plt.style.use('science')


def make_plot_output_rate(ax):
  linetypes = ['dashed', 'solid']
  
  # Define colors and config
  viridis = cm.get_cmap('viridis', 5)
  line_configs = {
      r'Bagheri2018': {
          'name': r'Bagheri 2018',
          'color': viridis(0),
          'line_style': 'solid',
          'legend_title': r'Real Devices',
      },
      r'ZMW_SOURCE': {
          'name': r'ZMW SOURCE',
          'color': viridis(1),
          'line_style': 'solid',
          'legend_title': r'Real Devices',
      },
      r'CoolerCondenser_Peeters2020': {
          'name': r'CoolerCond. (Peeters)',
          'color': viridis(2),
          'line_style': 'dotted',
          'legend_title': r'Upper Limit',
      },
      r'SMAG_Zhao2019': {
          'name': r'SMAG (Zhao)',
          'color': viridis(3),
          'line_style': 'dotted',
          'legend_title': r'Upper Limit',
      },
      r'Limits_Kim2020': {
          'name': r'Thermo Limited (Kim)',
          'color': viridis(4),
          'line_style': 'dotted',
          'legend_title': r'Upper Limit',
      },
      'charCurve_target1000mil_logistic': {
          'name': r'1.0 bil ppl (logistic)',
          'color': viridis(0),
          'line_style': 'dashed',
          #'legend_title': r'Characteristic Curves \\ Target @ 5 L/d/m\textsuperscript{2}',
          'legend_title': r'Char. Curves @ 5 L/d/m\textsuperscript{2}',
      },
      r'charCurve_target1000mil_linear': {
          'name': r'1.0 bil ppl (linear)',
          'color': viridis(1),
          'line_style': 'dashed',
          #'legend_title': r'Characteristic Curves \\ Target @ 5 L/d/m\textsuperscript{2}',
          'legend_title': r'Char. Curves @ 5 L/d/m\textsuperscript{2}',
      },
  }

  # Draw lines for fitted curves
  lines_handles = defaultdict(list)
  lines_labels_per_group = defaultdict(list)
  for line_label in lines_labels:
    line_conf = line_configs[line_label]
    line, = ax.plot(df_output_rate['yield'], df_output_rate[line_label] / 1e6,
                    linestyle=line_conf['line_style'], linewidth=1.5, color=line_conf['color'],
                    label=line_conf['name'])
    lines_handles[line_conf['legend_title']].append(line)
    lines_labels_per_group[line_conf['legend_title']].append(line_conf['name'])

  # Set the log scale on the y-axis with limits
  ax.set_xscale('log')
  ax.set_xlim([0.5, 100])
  ax.set_ylim([0, 2500])

  # Hide the top and right spines of the axis
  ax.spines['right'].set_visible(True)
  ax.spines['top'].set_visible(True)
  for spine in ax.spines.values():
    spine.set_linewidth(1.0)

  # Edit the major and minor ticks of the x and y axes
  ax.xaxis.set_tick_params(which='major', size=5, width=1.0, direction='in', top='on')
  ax.xaxis.set_tick_params(which='minor', size=2, width=0.75, direction='in', top='on')
  ax.yaxis.set_tick_params(which='major', size=5, width=1.0, direction='in', right='on')
  ax.yaxis.set_tick_params(which='minor', size=2, width=0.75, direction='in', right='on')

  # Format the x-axis ticks
  ax.xaxis.set_major_locator(mpl.ticker.FixedLocator([0.5, 1, 5, 10, 50, 100]))
  ax.set_xticklabels([0.5, 1, 5, 10, 50, 100])

  # Format y-axis ticks
  ax.yaxis.set_major_formatter(
      mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

  # Add the x-axis abd y-axis labels
  ax.set_xlabel(r'above Liters/day/m\textsuperscript{2} (log scale)', labelpad=10)
  ax.set_ylabel(r'People without SMDW [millions]', labelpad=10)

  # Add the legend for devices
  leg_title1 = sorted(lines_labels_per_group.keys())[1]
  leg1 = ax.legend(lines_handles[leg_title1],
                   lines_labels_per_group[leg_title1],
                   bbox_to_anchor=(0.5 - 0.55, -0.24), loc='upper center', ncol=1,
                   prop={'size': 8},
                   title=r'\ul{%s}' % leg_title1)

  # Add the legend for logistic lines
  leg_title2 = sorted(lines_labels_per_group.keys())[0]
  leg2 = ax.legend(lines_handles[leg_title2],
                   lines_labels_per_group[leg_title2],
                   bbox_to_anchor=(0.5 - 0.145, -0.24), loc='upper center', ncol=1,
                   prop={'size': 8},
                   title=r'\ul{%s}' % leg_title2)
  
  for text_artist in leg2.texts:
    text_artist.set_multialignment('center')
  
  # Add the legend for upper limits
  leg_title3 = sorted(lines_labels_per_group.keys())[2]
  leg3 = ax.legend(lines_handles[leg_title3],
                   lines_labels_per_group[leg_title3],
                   bbox_to_anchor=(0.5 + 0.325, -0.24), loc='upper center', ncol=1,
                   prop={'size': 8},
                   title=r'\ul{%s}' % leg_title3)
  
  # Manually add the first legend back
  ax.add_artist(leg1)
  ax.add_artist(leg2)
  ax.add_artist(leg3)


plt.rc('text.latex', preamble=r'\usepackage{soul}')
plt.rc('font', **{'family':'sans-serif', 'sans-serif': ['Arial']})
plt.rcParams["axes.axisbelow"] = False

with plt.style.context(['science', 'ieee', 'retro']):  # 'no-latex'
  fig, ax = plt.subplots()
  make_plot_output_rate(ax)
  fig.set_size_inches(4, 3)

fig.patch.set_facecolor('white')
fig.suptitle('User Reach above Output Rate by AWH')

# Load map images from the Drive

In [ ]:
import matplotlib.image as mpimg

# Read images
ghi_threshold_images = {}

ghi_threshold_images[(400, 10)] = mpimg.imread('gdrive/My Drive/GHI_400-rH_10.jpg')
ghi_threshold_images[(400, 30)] = mpimg.imread('gdrive/My Drive/GHI_400-rH_30.jpg')
ghi_threshold_images[(400, 50)] = mpimg.imread('gdrive/My Drive/GHI_400-rH_50.jpg')
ghi_threshold_images[(400, 70)] = mpimg.imread('gdrive/My Drive/GHI_400-rH_70.jpg')

ghi_threshold_images[(600, 10)] = mpimg.imread('gdrive/My Drive/GHI_600-rH_10.jpg')
ghi_threshold_images[(600, 30)] = mpimg.imread('gdrive/My Drive/GHI_600-rH_30.jpg')
ghi_threshold_images[(600, 50)] = mpimg.imread('gdrive/My Drive/GHI_600-rH_50.jpg')
ghi_threshold_images[(600, 70)] = mpimg.imread('gdrive/My Drive/GHI_600-rH_70.jpg')

# Combine Nature Plots

In [ ]:
%matplotlib inline

from collections import defaultdict

import matplotlib as mpl
from matplotlib import cm
from matplotlib import patheffects
import matplotlib.gridspec as gridspec

plt.style.reload_library()
plt.style.use('science')

plt.rc('text.latex', preamble=r'\usepackage{soul}')
plt.rcParams["axes.axisbelow"] = False
plt.rcParams['legend.title_fontsize'] = 6
plt.rcParams["legend.fontsize"] = 6
plt.rcParams["legend.handlelength"] = 1.2

with plt.style.context(['science', 'ieee', 'retro']):  # 'no-latex'
  fig, axes = plt.subplots(constrained_layout=False, figsize=(10, 10))

  gs0 = gridspec.GridSpec(1, 2, figure=fig, wspace=0.5)
  
  gs00 = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs0[0], wspace=0.25, hspace=0.75, height_ratios=[1.0, 6.0, 2.0])
  gs01 = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs0[1], wspace=0.25, hspace=0.95)

  gs000 = gridspec.GridSpecFromSubplotSpec(4, 2, subplot_spec=gs00[1], wspace=0.25, hspace=0.25)
  gs001 = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs00[2], wspace=0.25, hspace=0.95)

  ax = plt.subplot(gs01[0,0])
  make_plot_output_rate(ax)
  ax.set_title('User Reach by Output Rate')

  ax = plt.subplot(gs01[1,0])
  make_plot_specific_yields(ax)
  ax.set_title('User Reach by Specific Yield')

  axes = [plt.subplot(gs001[0,0]), plt.subplot(gs001[0,1])]
  make_plot_ghi(axes)

  # Add the images to the top-left subplots
  for ghi_idx, ghi in enumerate((400, 600)):
    for rh_idx, rh in enumerate((10, 30, 50, 70)):
      ax = plt.subplot(gs000[rh_idx,ghi_idx])
      ax.xaxis.set_tick_params(which='both', bottom=False, top=False, labelbottom=False, labeltop=False)
      ax.yaxis.set_tick_params(which='both', bottom=False, top=False, labelbottom=False, labeltop=False)
      ax.imshow(ghi_threshold_images[(ghi, rh)])
      if rh_idx == 0:
        ax.set_title(f'GHI [W/m²]: {int(ghi)}', fontsize=8)
      if ghi_idx == 0:
        ax.set_ylabel(f'rH {int(rh)}\%', fontsize=8)
        

fig.set_size_inches(6.5, 8.0)
fig.patch.set_facecolor('white')
#fig.suptitle('Figure 3')

In [ ]:
fig.savefig('combined_plots.svg')